# Maxwell unit: relaxation function & strain redistribution

***

<center><i>Petr Havlásek (c) 2023, petr.havlasek@cvut.cz</i></center>

***

In [7]:
online = True

if (online):
    import micropip
    await micropip.install('ipywidgets')

import math
import numpy as np    
    
import matplotlib.pyplot as plt
    
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display


ModuleNotFoundError: No module named 'micropip'

**Notation:**

>`E` ... spring stiffness [MPa]<br>

>`eta` ... dashpot viscosity [MPa day]<br>

>`tau` ... retardation time [day] (=`eta/E`)<br>

>`t` ... time of interest [day]<br>

>`tt` ... time of loading [day]<br>

## Definition of a relaxation for Maxwell unit

In [2]:
def Maxwell_R_func(E, eta, t, tt):

    tau = eta / E

    if t >= tt:
        dt = t-tt

        if tau > 0.:
            return E * math.exp(-(dt)/ tau) 
        else:
            return 0.
            
    else:
        return 0.

## Parameters definition

In [3]:
# reference values

# spring stiffness [MPa]
E_ref = 1.
# dashpot viscosity [MPa * day]
eta_ref = 1.
# reference relaxation time
tau_ref = eta_ref / E_ref

# variables ranges: powers of 10
log_E_min = -3
log_E_max = 3

log_eta_min = -3
log_eta_max = 3

log_tau_min = -2
log_tau_max = 2

# number of time steps
t_div = 100


## Plot relaxation function

In [4]:
def plot_Maxwell_R_func(E, eta, log_scale = True, show_tau = False, show_reference = True):

    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )

    ax.clear()

    # reference data with E_ref and eta_ref
    if show_reference:
        R_ref = []
        for t in times:
            R_ref.append( Maxwell_R_func(E_ref, eta_ref, t, 0.) )

        ax.plot(times, R_ref, lw=1.5, color="red", linestyle='dashed', label=r'$R_{ref}(t)$')
        
        if (show_tau):
            ax.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    R = []
    for t in times:
        R.append( Maxwell_R_func(E, eta, t, 0.) )
    ax.plot(times, R, lw=2., color="blue", label=r'$R(t)$')
    if (show_tau):
        ax.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax.set_xscale('log')
        ax.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax.set_xlim([0., 10**log_tau_max])


    ax.grid(True)
    ax.legend()
    ax.set_xlabel('Duration of loading, t-t\' [day]')
    ax.set_ylabel('Relaxation function, R [MPa]')
    
    display(fig)
    



## GUI for relaxation function

In [5]:
output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig)

E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')

aux = interactive(plot_Maxwell_R_func, E = E_slide, eta = eta_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)


NameError: name 'widgets' is not defined

## Response to prescribed and constant strain

In [22]:
def plot_Maxwell_strains(E, eta, epsilon, log_scale = True, show_tau = False, show_reference = True):

    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )

    ax2.clear()        

    # reference data with E_ref and eta_ref
    if show_reference:
        eps_e_ref = []
        eps_v_ref = []
        for t in times:
            R_ref = Maxwell_R_func(E_ref, eta_ref, t, 0.)
            sig_ref = epsilon * R_ref
            eps_e_ref.append( sig_ref / E_ref )
            eps_v_ref.append( epsilon - eps_e_ref[-1] )

        ax2.plot(times, eps_e_ref, lw=1.5, color="red", linestyle='dashed', label=r'$\varepsilon_{e,ref}(t)$')
        ax2.plot(times, eps_v_ref, lw=1.5, color="red", linestyle='solid', label=r'$\varepsilon_{v,ref}(t)$')
        
        if (show_tau):
            ax2.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    eps_e = []
    eps_v = []
    for t in times:
        R = Maxwell_R_func(E, eta, t, 0.)
        sig = epsilon * R
        eps_e.append( sig / E )
        eps_v.append( epsilon - eps_e[-1] )


    ax2.plot(times, eps_e, lw=2.0, color="blue", linestyle='dashed', label=r'$\varepsilon_{e}(t)$')
    ax2.plot(times, eps_v, lw=2.0, color="blue", linestyle='solid', label=r'$\varepsilon_{v}(t)$')
    
    if (show_tau):
        ax2.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax2.set_xscale('log')
        ax2.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax2.set_xlim([0., 10**log_tau_max])


    ax2.grid(True)
    ax2.legend()
    ax2.set_xlabel('Duration of loading, t-t\' [day]')
    ax2.set_ylabel(r'Strain, $\varepsilon$ [$10^{-6}$]')

    display(fig2)
    

## GUI for strain redistribution

In [23]:
output = widgets.Output()

fig2, ax2 = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig2)


E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')
epsilon_slide = widgets.FloatSlider(min=0., max=100., value=10., step=1., description='strain [$10^{-6}$]')

aux = interactive(plot_Maxwell_strains, E = E_slide, eta = eta_slide, epsilon = epsilon_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)

interactive(children=(FloatLogSlider(value=1.0, description='E [MPa]', max=3.0, min=-3.0, step=0.25), FloatLog…